In [9]:
from pulp import *
import pandas as pd

In [10]:
#Read data in to data frame (Only first 64 rows)
df = pd.read_excel("diet.xls", nrows=64)

#Check data 
df.head()
df.tail()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
59,Neweng Clamchwd,0.75,1 C (8 Fl Oz),175.7,10.0,5.0,1864.9,21.8,1.5,10.9,20.1,4.8,82.8,2.8
60,Tomato Soup,0.39,1 C (8 Fl Oz),170.7,0.0,3.8,1744.4,33.2,1.0,4.1,1393.0,133.0,27.6,3.5
61,"New E Clamchwd,W/Mlk",0.99,1 C (8 Fl Oz),163.7,22.3,6.6,992.0,16.6,1.5,9.5,163.7,3.5,186.0,1.5
62,"Crm Mshrm Soup,W/Mlk",0.65,1 C (8 Fl Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6
63,"Beanbacn Soup,W/Watr",0.67,1 C (8 Fl Oz),172.0,2.5,5.9,951.3,22.8,8.6,7.9,888.0,1.5,81.0,2.0


In [11]:
#Create a list of all foods
food_list = list(df['Foods'])


#Create a dictionary of price (all foods)
price = dict(zip(food_list , df['Price/ Serving']))


#Create a list of nutrition facts names and min and max of each one
col_name = list(df.columns[3:])
var_name = ['calories', 'cholesterol', 'fat', 'sodium', 
            'carbohydrates', 'fiber', 'protein', 'vit_A', 'vit_C', 
            'calcium', 'iron']
val_min = [1500,  30, 20,  800, 130, 125,  60,  1000,  400,  700, 10]
val_max = [2500, 240, 70, 2000, 450, 250, 100, 10000, 5000, 1500, 40]


#Create a list of nutrition_facts list that containes a dictionary of foods and it's nutrition(calories, cholesterol, ...)
nutrition_facts = []
for i,j in enumerate(col_name):
    nutrition_facts.append(dict(zip(food_list , df[col_name[i]])))

In [12]:
#Define variables
food_vars = LpVariable.dicts("Food", food_list, lowBound=0, cat=LpContinuous)
food_vars_included = LpVariable.dicts("Food_included", food_list, lowBound=0, upBound=1, cat=LpBinary)

In [13]:
#Create Objective function (Minimization)
problem = LpProblem("Food_Cost", LpMinimize)
problem += lpSum([price[food] * food_vars[food] for food in food_list])

In [14]:
#Add Constraints
for i,n_name in enumerate(col_name):
        problem += lpSum(nutrition_facts[i][food] * food_vars[food] for food in food_list) >= val_min[i], 'Minimum ' + n_name
        problem += lpSum(nutrition_facts[i][food] * food_vars[food] for food in food_list) <= val_max[i], 'Maximum ' + n_name
        
        
#part a
#If a food is selected, then a minimum of 1/10 serving must be chosen
for food in food_list:
    problem += food_vars[food] >= food_vars_included[food] * 0.1
    problem += food_vars[food] <= food_vars_included[food] * (val_max[0] / nutrition_facts[0][food]) #x <= y * MAX_calories
    
#part b
#Many people dislike celery and frozen broccoli. So at most one, but not both, can be selected
problem += 1 >= food_vars_included['Frozen Broccoli'] + food_vars_included['Celery, Raw']


In [15]:
#Solve the optimization problem
problem.solve()

1

In [16]:
for var in problem.variables():
    if var.varValue > 0:
        print(var.name, "==>", var.varValue)

Food_Celery,_Raw ==> 43.154119
Food_Lettuce,Iceberg,Raw ==> 80.919121
Food_Oranges ==> 3.0765161
Food_Peanut_Butter ==> 2.0464575
Food_Poached_Eggs ==> 0.14184397
Food_Popcorn,Air_Popped ==> 13.181772
Food_included_Celery,_Raw ==> 1.0
Food_included_Lettuce,Iceberg,Raw ==> 1.0
Food_included_Oranges ==> 1.0
Food_included_Peanut_Butter ==> 1.0
Food_included_Poached_Eggs ==> 1.0
Food_included_Popcorn,Air_Popped ==> 1.0


In [17]:
print(value(problem.objective))

4.4878950176
